In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

/home/vipuser/anaconda3/envs/GLM4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = '/home/LLM_para/para_glm4'
lora_path = './final_checkpoint_GLM4_Table_SFT_cropped'
# val_dataset_path = "/home/code/chat_SQL/Dataset/GLM4/ENG_ori/My_idea_dataset/T2Q_GLM4_SFT_val_table.jsonl"

In [3]:
model = AutoPeftModelForCausalLM.from_pretrained(
        lora_path, trust_remote_code= True, device_map='auto')

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████| 10/10 [01:36<00:00,  9.66s/it]


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def filter_descibe(sql_statement):
    modified_sql = re.sub(r'(\w+)\s+(INTEGER|int|NUMERIC|INT|REAL|TEXT|DATETIME|datetime|bigint\(?\d*\)?|CHAR\(?\d*\)?|char\(?\d*\)?|varchar2\(?\d*\)|VARCHAR\(?\d*\)|varchar\(?\d*\)|DECIMAL\(?\d*,\s*\d*\)?)', r'\1', sql_statement)
    return  modified_sql

In [6]:
from tqdm import tqdm
import copy
import re

# 获取REF表数据
tab_str = ""
count = 0
df = pd.read_csv("./mydataset_new/table_schema_Reference_cropped.csv", encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['Reference_group']
    table_group = filter_descibe(table_group)
    table_group = table_group.replace("\r","")
    table_group = table_group.replace("\n","")
    table_group = table_group.replace("  "," ")
    tab_str += table_group + "\n"
    
df = pd.read_csv("./mydataset_new/table_schema_noReference_cropped.csv", encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['noReference_group']
    table_group = filter_descibe(table_group)
    table_group = table_group.replace("\r", "")
    table_group = table_group.replace("\n", "")
    table_group = table_group.replace("  ", " ")
    tab_str += table_group + "\n"

schema_str = tab_str.strip()
check_token = tokenizer(f"{schema_str}", add_special_tokens=False)
print(len(check_token["input_ids"]))
max_token = 5200

100%|████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 10137.70it/s]

4900


In [7]:
def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
#     print(response)
     
    return response

# instruction = row['instruction']
# input_value = row['input']
def get_instruction(schema):
    instruction = f" I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{schema}"
    return instruction

def get_input_value(question):
    input_value = f"{question}"
    return input_value

In [8]:
import re
df = pd.read_csv("./mydataset_new/validation_dataset_formatted_cropped.csv",encoding="utf-8")
outer_index = 0
import re
import time
start_time = time.time() 
pattern = re.compile(r'-- Tables: ([\w, ]+);')
results = []
instruction = get_instruction(schema_str)
#  schema_slicing
for index, row in tqdm(df.iterrows(), total=len(df)):
    question = row['question']
    ref_tables = row['correct_tables']
    messages = [
    {"role": "system", "content": f"{instruction}"},
    {"role": "user", "content": f"{question}"}]
    response = predict(messages, model, tokenizer)
    match = re.search(pattern, response)
    if match:
    # 提取匹配的表名字符串
        table_names_str = match.group(1)

        # 使用中文逗号分隔表名，这里我们替换中文逗号为英文逗号并去除空格
    #         table_names = [name.strip() for name in table_names_str.replace('，', ',').split(',')]
        table_names = table_names_str.split(', ')
        # 将table_names整合成一个由英文逗号分隔的字符串
        predict_table = ', '.join(table_names)
        print(f"++++++++++++++++index:{index}++++++++++++++++++++++++")
        print("预测表名：", predict_table)
        print("正确表名：", ref_tables)
        print("############################################")
    else:
        predict_table = "None"
    results.append([ref_tables,predict_table])
end_time = time.time() 
print(len(predict_table))  

  1%|▍                                                                                  | 1/178 [00:04<12:01,  4.08s/it]

++++++++++++++++index:0++++++++++++++++++++++++
预测表名： products, manufacturers
正确表名： products, manufacturers
############################################


  1%|▉                                                                                  | 2/178 [00:05<06:51,  2.34s/it]

++++++++++++++++index:1++++++++++++++++++++++++
预测表名： products
正确表名： products
############################################


  2%|█▍                                                                                 | 3/178 [00:06<05:11,  1.78s/it]

...

++++++++++++++++index:175++++++++++++++++++++++++
预测表名： railway
正确表名： railway
############################################


 99%|████████████████████████████████████████████████████████████████████████████████▌| 177/178 [03:36<00:01,  1.15s/it]

++++++++++++++++index:176++++++++++++++++++++++++
预测表名： book
正确表名： book
############################################


100%|█████████████████████████████████████████████████████████████████████████████████| 178/178 [03:37<00:00,  1.22s/it]

++++++++++++++++index:177++++++++++++++++++++++++
预测表名： book, publication
正确表名： book, publication
############################################
17


In [9]:
new_df = pd.DataFrame(results, columns = ['reference_tables','predicted_tables'])

In [10]:
total_samples = len(new_df)
total_accuracy = 0
filtered_accuracy = 0
total_precision = 0
total_recall = 0
count = 0
for index, row in new_df.iterrows():
#     count+=1
#     print("index:",count)
    if not row['predicted_tables'] or pd.isna(row['predicted_tables']):
        continue
    predicted_tables = row['predicted_tables'].split(",")
    reference_tables = row['reference_tables'].split(",")
    
    # Convert to lowercase and strip whitespace for comparison
    predicted_tables = [x.lower().replace("--","").replace("**","").strip() for x in predicted_tables]
    reference_tables = [x.lower().strip() for x in reference_tables]
    
    # Calculate accuracy
    if set(predicted_tables) == set(reference_tables):
        total_accuracy += 1

    # Calculate precision and recall
    true_positives = len(set(predicted_tables) & set(reference_tables))
    false_positives = len(set(predicted_tables) - set(reference_tables))
    false_negatives = len(set(reference_tables) - set(predicted_tables))

    if true_positives == len(reference_tables):
        filtered_accuracy += 1
    
    if len(predicted_tables) > 0:
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
    
    total_precision += precision
    total_recall += recall

# Calculate average precision and recall
avg_precision = total_precision / total_samples
avg_recall = total_recall / total_samples

# Calculate total accuracy
accuracy = total_accuracy / total_samples
filtered_accuracy = filtered_accuracy / total_samples

print("Total Accuracy:", accuracy)
print("Filtered Accuracy:", filtered_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)

Total Accuracy: 0.9606741573033708
Filtered Accuracy: 0.9775280898876404
Average Precision: 0.99063670411985
Average Recall: 0.9920411985018726


In [11]:
total_data = len(df['question'])
total_time = end_time - start_time
# print(total_data)
signal_infernece_time = total_time/total_data
print(signal_infernece_time)

1.2234598384814317


In [13]:
new_df.to_csv("pre_DTS_SQL_table.csv", encoding="utf-8")